# How post-game LoL stats affect the outcome of the game

### Description

[League of Legends](https://en.wikipedia.org/wiki/League_of_Legends) is a multiple online battle arena (MOBA) video game that involves two teams of five players. Each team wins by destroying the enemy "nexus" located in the enemy base. Each player earns experience points (exp) and gold to give their character extra statistics in order to defeat the enemy. 

In this project, I will be classifying League of Legends games as either `win` or `loss` based on player game statistics at the end of the game, such as number of kills/deaths, gold earned, and damage dealt. I will explore how these different statistics in the game affect the outcome of these games. In particular, I will be looking at my own ranked solo/duo games under the summoner name `Mirinda`.  

I will be obtaining my data through the [RiotAPI](https://developer.riotgames.com/apis). I will pull a list of matches and choose a particular player to view their statistics.

In order to explore how different post-game statistics affect the outcome of the game, I will train multiple classifications models using different sets of features. The classification models are as follows:

- Perceptron
- Logistic regression
- Decision tree
- Random forest
- Support vector machine with radial basis function kernel


In each of these models, I will train 10 different models for each set of features. I will then test each model with 10 sets of testing data. I will record the accuracy of the trained models for different sets of features and compare the overall averages of each different model for each set of features.

In [1]:
# imports
import csv 
import numpy as np
import os
import pandas as pd
from tabulate import tabulate

### The data

I have obtained my data through the RiotAPI using a Python script with the `riotwatcher` package.  I pulled a list of 907 matches. Through each match, I iterated through each `participant` and pulled game results for `Mirinda`. I then wrote this data obtained onto a CSV file. 

Each match contains a list of 54 different statistics (see`df.columns`). These will be used as possible features.

In [2]:
# read in data
df = pd.read_csv('game-data.csv',
                     encoding='utf-8')
# possible features exluding `win` and `matchID`
df.columns

Index(['assists', 'bountyLevel', 'champExperience', 'champLevel',
       'consumablesPurchased', 'damageDealtToBuildings',
       'damageDealtToObjectives', 'damageDealtToTurrets',
       'damageSelfMitigated', 'deaths', 'detectorWardsPlaced', 'doubleKills',
       'goldEarned', 'goldSpent', 'killingSprees', 'kills',
       'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill',
       'longestTimeSpentLiving', 'magicDamageDealt',
       'magicDamageDealtToChampions', 'magicDamageTaken',
       'neutralMinionsKilled', 'pentaKills', 'physicalDamageDealt',
       'physicalDamageDealtToChampions', 'physicalDamageTaken', 'quadraKills',
       'sightWardsBoughtInGame', 'spell1Casts', 'spell2Casts', 'spell3Casts',
       'spell4Casts', 'timeCCingOthers', 'timePlayed', 'totalDamageDealt',
       'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates',
       'totalDamageTaken', 'totalHeal', 'totalHealsOnTeammates',
       'totalMinionsKilled', 'totalTimeCCDealt', 'totalTim

### Feature selections

There are 7 sets of features I will be using to compare with one another:

1. all features
2. player combat -- offensive: 'largestCriticalStrike', 'magicDamageDealt','magicDamageDealtToChampions', 'physicalDamageDealt', 'physicalDamageDealtToChampions', 'totalDamageDealt', 'totalDamageDealtToChampions'
3. player combat -- defensive: 'damageSelfMitigated', 'magicDamageTaken', 'physicalDamageTaken', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 'totalHeal', 'totalHealsOnTeammates', 'trueDamageTaken'
4. player combat -- offensive AND defensive
5. spell casts: 'spell1Casts', 'spell2Casts', 'spell3Casts', 'spell4Casts'
6. KDA, kill streaks: 'assists', 'bountyLevel', 'deaths', 'doubleKills', 'killingSprees', 'kills', 'largestKillingSpree', 'largestMultiKill', 'pentaKills', 'quadraKills', 'tripleKills'
7. vision: 'detectorWardsPlaced', 'sightWardsBoughtInGame', 'visionScore', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced'

In [3]:
# features: all
X_all = df.iloc[:, :54]
X_all = np.array(X_all)

# features: player combat -- offensive
# 'largestCriticalStrike', 'magicDamageDealt','magicDamageDealtToChampions', 'physicalDamageDealt',
# 'physicalDamageDealtToChampions', 'totalDamageDealt', 'totalDamageDealtToChampions'
# 'trueDamageDealt', 'trueDamageDealtToChampions',
X_pcombatOff = df.iloc[:, [16, 20, 21, 25, 26, 36, 37, 47, 48]]
X_pcombatOff = np.array(X_pcombatOff)

# features: player combat -- defensive
# 'damageSelfMitigated', 'magicDamageTaken', 'physicalDamageTaken', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
# 'totalHeal', 'totalHealsOnTeammates', 'trueDamageTaken'
X_pcombatDef = df.iloc[:, [8, 22, 27, 38, 39, 40, 41, 49]]
X_pcombatDef = np.array(X_pcombatDef)

# features: player combat -- offensive AND defensive
X_pcombat = df.iloc[:, [16, 20, 21, 25, 26, 36, 37, 47, 48, 8, 22, 27, 38, 39, 40, 41, 49]]
X_pcombat = np.array(X_pcombat)

# features: spell casts
# 'spell1Casts', 'spell2Casts', 'spell3Casts','spell4Casts'
X_spells = df.iloc[:, [30, 31, 32, 33]]
X_spells = np.array(X_spells)

# features: KDA, kill streaks
# 'assists', 'bountyLevel', 'deaths', 'doubleKills', 'killingSprees', 'kills',
# 'largestKillingSpree', 'largestMultiKill', 'pentaKills', 'quadraKills', 'tripleKills'
X_kda = df.iloc[:, [0, 1, 9, 11, 14, 15, 17, 18, 24, 28, 46]]
X_kda = np.array(X_kda)

# features: vision
# 'detectorWardsPlaced', 'sightWardsBoughtInGame', 'visionScore', 'visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced'
X_vision = df.iloc[:, [10, 29, 50, 51, 52, 53]]
X_vision = np.array(X_vision)

# list of different sets of features
features_list = [X_all, X_pcombatOff, X_pcombatDef, X_pcombat, X_spells, X_kda, X_vision]

# classes
y = df.iloc[:, 54].values
# True = 1, False = 0
y = np.where(y, 1, 0)

### Split data into training/testing sets

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

"""
split_data(): split data into n sets of training and testing sets with a 70/30 split respectively, and standardize training set
    input:
        randomStart - start value
        randomEnd - end value
        features - set of features as an array
        targets - classes
    output:
        dictionary of data splits:
            dataSplits[random_state] = [X_train_std, X_test_std, y_train, y_test]
"""

def split_data(randomStart, randomEnd, features, targets):

    # dataSplits[random_state] = [X_train_std, X_test_std, y_train, y_test]
    dataSplits = {}
    
    for i in range(randomStart, randomEnd):
        # Split data into testing and training 
        X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, random_state=i, stratify=targets)

        # Standardize training set
        sc = StandardScaler()
        sc.fit(X_train)
        X_train_std = sc.transform(X_train)
        X_test_std = sc.transform(X_test)

        # add split to dictionary
        dataSplits[i] = [X_train_std, X_test_std, y_train, y_test]
        
    return dataSplits

In [5]:
# Split data into training and testing
X_all_split = split_data(1, 11, X_all, y)
X_pcombatOff_split = split_data(1, 11, X_pcombatOff, y)
X_pcombatDef_split = split_data(1, 11, X_pcombatDef, y)
X_pcombat_split = split_data(1, 11, X_pcombat, y)
X_spells_split = split_data(1, 11, X_spells, y)
X_kda_split = split_data(1, 11, X_kda, y)
X_vision_split = split_data(1, 11, X_vision, y)

### Train Perceptron models



In [6]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

"""
train_perceptrons(): train n different perceptron models and test each n times
    input:
        randomStart - start value
        randomEnd - end value
        dataSplits - dictionary of data splits
    output:
        dictionary of perceptron models:
            ppns[Perceptron model] = [(Misclassified examples, Accuracy)...]
"""

def train_perceptrons(randomStart, randomEnd, dataSplits):
        
    # ppns[Perceptron model] = [(Misclassified examples, Accuracy)...]
    ppns = {}

    for randState in dataSplits:

        # current testing and training split
        currentSplit = dataSplits[randState]

        # list of models for each random state
        results = []

        # Train Perceptron model
        ppn = Perceptron(eta0=0.1, random_state=randState)
        ppn.fit(currentSplit[0], currentSplit[2]) # X_train_std, y_train

        # test Perceptron model x times
        for i in range(randomStart, randomEnd):

            # put results into dict
            y_pred = ppn.predict(dataSplits[i][1]) # X_test_std

            # append tuple (Misclassified examples, Accuracy) ==> (y_test != y_pred).sum(), accuracy_score(y_test, y_pred))
            results.append(((dataSplits[i][3] != y_pred).sum(), accuracy_score(dataSplits[i][3], y_pred)))
            
        # append results to current perceptron dict
        ppns[ppn] = results
        
    return ppns

In [7]:
"""
print_avgs(): print average misclassified examples and average accuracy
    input:
        model_dict - dictionary of models:
            model_dict[model] = [(Misclassified examples, Accuracy)...]
    output:
        array containing average misclassified examples and average accuracy:
            [average misclassified examples, average accuracy]
"""
def print_avgs(model_dict):
    
    total = len(model_dict)*len(model_dict[list(model_dict.keys())[0]])
    sums = []
    
    for model in model_dict:
        results = model_dict[model]
        sums.append([sum(tup) for tup in zip(*results)])

    final_list = [sum(value) for value in zip(*sums)]
    final_list = [x / total for x in final_list]
    
    print('Avg misclassified examples: %d' % final_list[0])
    print('Avg accuracy: %.3f' % final_list[1])
    
    return final_list

### Train Perceptrons

In [8]:
# train Perceptrons on all features
ppns_all = train_perceptrons(1, 11, X_all_split)
ppns_all_results = print_avgs(ppns_all)

Avg misclassified examples: 25
Avg accuracy: 0.906


In [9]:
# train Perceptrons on player combat -- offensive
ppns_pcombatOff = train_perceptrons(1, 11, X_pcombatOff_split)
ppns_pcombatOff_results = print_avgs(ppns_pcombatOff)

Avg misclassified examples: 125
Avg accuracy: 0.538


In [10]:
# train Perceptrons on player combat -- defensive
ppns_pcombatDef = train_perceptrons(1, 11, X_pcombatDef_split)
ppns_pcombatDef_results = print_avgs(ppns_pcombatDef)

Avg misclassified examples: 94
Avg accuracy: 0.652


In [11]:
# train Perceptrons on player combat -- both
ppns_pcombat = train_perceptrons(1, 11, X_pcombat_split)
ppns_pcombat_results = print_avgs(ppns_pcombat)

Avg misclassified examples: 78
Avg accuracy: 0.712


In [12]:
# train Perceptrons on spell casts
ppns_spells = train_perceptrons(1, 11, X_spells_split)
ppns_spells_results = print_avgs(ppns_spells)

Avg misclassified examples: 131
Avg accuracy: 0.517


In [13]:
# train Perceptrons on kda/killing sprees
ppns_kda = train_perceptrons(1, 11, X_kda_split)
ppns_kda_results = print_avgs(ppns_kda)

Avg misclassified examples: 51
Avg accuracy: 0.812


In [14]:
# train Perceptrons on vision
ppns_vision = train_perceptrons(1, 11, X_vision_split)
ppns_vision_results = print_avgs(ppns_vision)

Avg misclassified examples: 118
Avg accuracy: 0.566


### Perceptron results

In [15]:
features_about =  ["All features", "Player combat (offensive)", "Player combat (defensive)",
                  "Player combat", "Spells casted", "KDA/kill streaks", "Vision"]
ppn_results_list = [ppns_all_results, ppns_pcombatOff_results, ppns_pcombatDef_results, 
                  ppns_pcombat_results, ppns_spells_results, ppns_kda_results, ppns_vision_results]

ppn_results = pd.DataFrame(ppn_results_list, index = features_about, columns = ["Avg misclassified examples", "Avg accuracy"])
ppn_results = ppn_results.sort_values(by=["Avg accuracy"], ascending=False)
ppn_results

,Avg misclassified examples,Avg accuracy
All features,25.54,0.906447
KDA/kill streaks,51.45,0.811538
Player combat,78.53,0.712344
Player combat (defensive),94.95,0.652198
Vision,118.48,0.566007
Player combat (offensive),125.99,0.538498
Spells casted,131.79,0.517253


### Train logistic regression

In [36]:
from sklearn.linear_model import LogisticRegression

"""
train_lrGDs(): train n different logistic regression GD models and test each n times
    input:
        randomStart - start value
        randomEnd - end value
        dataSplits - dictionary of data splits
    output:
        dictionary of logistic regression GD models:
            lrGDs[logistic regression GD model] = [(Misclassified examples, Accuracy)...]
"""
def train_lrGDs(randomStart, randomEnd, dataSplits):
        
    # lrGDs[logistic regression GD model] = [(Misclassified examples, Accuracy)...]
    lrGDs = {}

    for randState in dataSplits:

        # current testing and training split
        currentSplit = dataSplits[randState]

        # list of models for each random state
        results = []

        # Train logistic regression GD model
        lrGD = LogisticRegression(random_state=randState)
        lrGD.fit(currentSplit[0], currentSplit[2]) # X_train_std, y_train

        # test logistic regression GD model x times
        for i in range(randomStart, randomEnd):

            # put results into dict
            y_pred = lrGD.predict(dataSplits[i][1]) # X_test_std

            # append tuple (Misclassified examples, Accuracy) ==> (y_test != y_pred).sum(), accuracy_score(y_test, y_pred))
            results.append(((dataSplits[i][3] != y_pred).sum(), accuracy_score(dataSplits[i][3], y_pred)))
            
        # append results to current perceptron dict
        lrGDs[lrGD] = results
        
    return lrGDs

In [37]:
# train logistic regression GD on all features
lrGDs_all = train_lrGDs(1, 11, X_all_split)
lrGDs_all_results = print_avgs(lrGDs_all)

Avg misclassified examples: 16
Avg accuracy: 0.941


In [38]:
# train logistic regression GD on player combat -- offensive
lrGDs_pcombatOff = train_lrGDs(1, 11, X_pcombatOff_split)
lrGDs_pcombatOff_results = print_avgs(lrGDs_pcombatOff)

Avg misclassified examples: 115
Avg accuracy: 0.578


In [39]:
# train logistic regression GD on player combat -- defensive
lrGDs_pcombatDef = train_lrGDs(1, 11, X_pcombatDef_split)
lrGDs_pcombatDef_results = print_avgs(lrGDs_pcombatDef)

Avg misclassified examples: 64
Avg accuracy: 0.762


In [40]:
# train logistic regression GD on player combat -- both
lrGDs_pcombat = train_lrGDs(1, 11, X_pcombat_split)
lrGDs_pcombat_results = print_avgs(lrGDs_pcombat)

Avg misclassified examples: 59
Avg accuracy: 0.782


In [41]:
# train logistic regression GD on spell casts
lrGDs_spells = train_lrGDs(1, 11, X_spells_split)
lrGDs_spells_results = print_avgs(lrGDs_spells)

Avg misclassified examples: 117
Avg accuracy: 0.570


In [42]:
# train logistic regression GD on kda/killing sprees
lrGDs_kda = train_lrGDs(1, 11, X_kda_split)
lrGDs_kda_results = print_avgs(lrGDs_kda)

Avg misclassified examples: 36
Avg accuracy: 0.866


In [43]:
# train logistic regression GD on vision
lrGDs_vision = train_lrGDs(1, 11, X_vision_split)
lrGDs_vision_results = print_avgs(lrGDs_vision)

Avg misclassified examples: 95
Avg accuracy: 0.649


### Logistic regression GD results

In [44]:
lrGD_results_list = [lrGDs_all_results, lrGDs_pcombatOff_results, lrGDs_pcombatDef_results, 
                  lrGDs_pcombat_results, lrGDs_spells_results, lrGDs_kda_results, lrGDs_vision_results]

lrGD_results = pd.DataFrame(lrGD_results_list, index = features_about, columns = ["Avg misclassified examples", "Avg accuracy"])
lrGD_results = lrGD_results.sort_values(by=["Avg accuracy"], ascending=False)
lrGD_results

,Avg misclassified examples,Avg accuracy
All features,16.09,0.941062
KDA/kill streaks,36.55,0.866117
Player combat,59.44,0.782271
Player combat (defensive),64.87,0.762381
Vision,95.80,0.649084
Player combat (offensive),115.30,0.577656
Spells casted,117.45,0.569780


### Train decision tree

In [45]:
from sklearn.tree import DecisionTreeClassifier

"""
train_dts(): train n different decision tree models and test each n times
    input:
        randomStart - start value
        randomEnd - end value
        dataSplits - dictionary of data splits
    output:
        dictionary of decision tree models:
            dts[decision tree model] = [(Misclassified examples, Accuracy)...]
"""
def train_dts(randomStart, randomEnd, dataSplits):
        
    # dts[decision tree model] = [(Misclassified examples, Accuracy)...]
    dts = {}

    for randState in dataSplits:

        # current testing and training split
        currentSplit = dataSplits[randState]

        # list of models for each random state
        results = []

        # Train decision tree model
        dt = DecisionTreeClassifier(random_state=randState)
        dt.fit(currentSplit[0], currentSplit[2]) # X_train_std, y_train

        # test decision tree model x times
        for i in range(randomStart, randomEnd):

            # put results into dict
            y_pred = dt.predict(dataSplits[i][1]) # X_test_std

            # append tuple (Misclassified examples, Accuracy) ==> (y_test != y_pred).sum(), accuracy_score(y_test, y_pred))
            results.append(((dataSplits[i][3] != y_pred).sum(), accuracy_score(dataSplits[i][3], y_pred)))
            
        # append results to current perceptron dict
        dts[dt] = results
        
    return dts

In [47]:
# train decision tree on all features
dts_all = train_dts(1, 11, X_all_split)
dts_all_results = print_avgs(dts_all)

Avg misclassified examples: 23
Avg accuracy: 0.916


In [48]:
# train decision tree on player combat -- offensive
dts_pcombatOff = train_dts(1, 11, X_pcombatOff_split)
dts_pcombatOff_results = print_avgs(dts_pcombatOff)

Avg misclassified examples: 88
Avg accuracy: 0.674


In [49]:
# train decision tree on player combat -- defensive
dts_pcombatDef = train_dts(1, 11, X_pcombatDef_split)
dts_pcombatDef_results = print_avgs(dts_pcombatDef)

Avg misclassified examples: 56
Avg accuracy: 0.794


In [50]:
# train decision tree on player combat -- both
dts_pcombat = train_dts(1, 11, X_pcombat_split)
dts_pcombat_results = print_avgs(dts_pcombat)

Avg misclassified examples: 55
Avg accuracy: 0.796


In [51]:
# train decision tree on spell casts
dts_spells = train_dts(1, 11, X_spells_split)
dts_spells_results = print_avgs(dts_spells)

Avg misclassified examples: 79
Avg accuracy: 0.710


In [52]:
# train decision tree on kda/killing sprees
dts_kda = train_dts(1, 11, X_kda_split)
dts_kda_results = print_avgs(dts_kda)

Avg misclassified examples: 26
Avg accuracy: 0.903


In [53]:
# train decision tree on vision
dts_vision = train_dts(1, 11, X_vision_split)
dts_vision_results = print_avgs(dts_vision)

Avg misclassified examples: 55
Avg accuracy: 0.796


### Decision tree results

In [54]:
dt_results_list = [dts_all_results, dts_pcombatOff_results, dts_pcombatDef_results, 
                  dts_pcombat_results, dts_spells_results, dts_kda_results, dts_vision_results]

dt_results = pd.DataFrame(dt_results_list, index = features_about, columns = ["Avg misclassified examples", "Avg accuracy"])
dt_results = dt_results.sort_values(by=["Avg accuracy"], ascending=False)
dt_results

,Avg misclassified examples,Avg accuracy
All features,23.00,0.915751
KDA/kill streaks,26.41,0.903260
Vision,55.60,0.796337
Player combat,55.67,0.796081
Player combat (defensive),56.20,0.794139
Spells casted,79.21,0.709853
Player combat (offensive),88.95,0.674176


### Train random forest

In [55]:
from sklearn.ensemble import RandomForestClassifier

"""
train_rfs(): train n different random forest models and test each n times
    input:
        randomStart - start value
        randomEnd - end value
        dataSplits - dictionary of data splits
    output:
        dictionary of random forest models:
            rfs[random forest model] = [(Misclassified examples, Accuracy)...]
"""
def train_rfs(randomStart, randomEnd, dataSplits):
        
    # rfs[random forest model] = [(Misclassified examples, Accuracy)...]
    rfs = {}

    for randState in dataSplits:

        # current testing and training split
        currentSplit = dataSplits[randState]

        # list of models for each random state
        results = []

        # Train random forest model
        rf = RandomForestClassifier(random_state=randState)
        rf.fit(currentSplit[0], currentSplit[2]) # X_train_std, y_train

        # test random forest model x times
        for i in range(randomStart, randomEnd):

            # put results into dict
            y_pred = rf.predict(dataSplits[i][1]) # X_test_std

            # append tuple (Misclassified examples, Accuracy) ==> (y_test != y_pred).sum(), accuracy_score(y_test, y_pred))
            results.append(((dataSplits[i][3] != y_pred).sum(), accuracy_score(dataSplits[i][3], y_pred)))
            
        # append results to current perceptron dict
        rfs[rf] = results
        
    return rfs

In [56]:
# train random forest on all features
rfs_all = train_rfs(1, 11, X_all_split)
rfs_all_results = print_avgs(rfs_all)

Avg misclassified examples: 12
Avg accuracy: 0.955


In [57]:
# train random forest on player combat -- offensive
rfs_pcombatOff = train_rfs(1, 11, X_pcombatOff_split)
rfs_pcombatOff_results = print_avgs(rfs_pcombatOff)

Avg misclassified examples: 61
Avg accuracy: 0.775


In [58]:
# train random forest on player combat -- defensive
rfs_pcombatDef = train_rfs(1, 11, X_pcombatDef_split)
rfs_pcombatDef_results = print_avgs(rfs_pcombatDef)

Avg misclassified examples: 35
Avg accuracy: 0.871


In [59]:
# train random forest on player combat -- both
rfs_pcombat = train_rfs(1, 11, X_pcombat_split)
rfs_pcombat_results = print_avgs(rfs_pcombat)

Avg misclassified examples: 30
Avg accuracy: 0.890


In [60]:
# train random forest on spell casts
rfs_spells = train_rfs(1, 11, X_spells_split)
rfs_spells_results = print_avgs(rfs_spells)

Avg misclassified examples: 63
Avg accuracy: 0.767


In [61]:
# train random forest on kda/killing sprees
rfs_kda = train_rfs(1, 11, X_kda_split)
rfs_kda_results = print_avgs(rfs_kda)

Avg misclassified examples: 23
Avg accuracy: 0.914


In [62]:
# train random forest on vision
rfs_vision = train_rfs(1, 11, X_vision_split)
rfs_vision_results = print_avgs(rfs_vision)

Avg misclassified examples: 44
Avg accuracy: 0.836


### Random forest results

In [63]:
rf_results_list = [rfs_all_results, rfs_pcombatOff_results, rfs_pcombatDef_results, 
                  rfs_pcombat_results, rfs_spells_results, rfs_kda_results, rfs_vision_results]

rf_results = pd.DataFrame(rf_results_list, index = features_about, columns = ["Avg misclassified examples", "Avg accuracy"])
rf_results = rf_results.sort_values(by=["Avg accuracy"], ascending=False)
rf_results

,Avg misclassified examples,Avg accuracy
All features,12.20,0.955311
KDA/kill streaks,23.59,0.913590
Player combat,30.03,0.890000
Player combat (defensive),35.16,0.871209
Vision,44.86,0.835678
Player combat (offensive),61.53,0.774615
Spells casted,63.69,0.766703


### Train support vector machine (SVM) with radial basis function (rbf) kernel

In [64]:
from sklearn.svm import SVC

"""
train_svms(): train n different SVM models and test each n times
    input:
        randomStart - start value
        randomEnd - end value
        dataSplits - dictionary of data splits
    output:
        dictionary of SVM models:
            SVMs[SVM model] = [(Misclassified examples, Accuracy)...]
"""
def train_svms(randomStart, randomEnd, dataSplits):
        
    # svms[SVM model] = [(Misclassified examples, Accuracy)...]
    svms = {}

    for randState in dataSplits:

        # current testing and training split
        currentSplit = dataSplits[randState]

        # list of models for each random state
        results = []

        # Train random forest model
        svm = SVC(random_state=randState)
        svm.fit(currentSplit[0], currentSplit[2]) # X_train_std, y_train

        # test random forest model x times
        for i in range(randomStart, randomEnd):

            # put results into dict
            y_pred = svm.predict(dataSplits[i][1]) # X_test_std

            # append tuple (Misclassified examples, Accuracy) ==> (y_test != y_pred).sum(), accuracy_score(y_test, y_pred))
            results.append(((dataSplits[i][3] != y_pred).sum(), accuracy_score(dataSplits[i][3], y_pred)))
            
        # append results to current perceptron dict
        svms[svm] = results
        
    return svms

In [65]:
# train SVM on all features
svms_all = train_svms(1, 11, X_all_split)
svms_all_results = print_avgs(svms_all)

Avg misclassified examples: 19
Avg accuracy: 0.930


In [66]:
# train SVM on player combat -- offensive
svms_pcombatOff = train_svms(1, 11, X_pcombatOff_split)
svms_pcombatOff_results = print_avgs(svms_pcombatOff)

Avg misclassified examples: 104
Avg accuracy: 0.616


In [67]:
# train SVM on player combat -- defensive
svms_pcombatDef = train_svms(1, 11, X_pcombatDef_split)
svms_pcombatDef_results = print_avgs(svms_pcombatDef)

Avg misclassified examples: 65
Avg accuracy: 0.761


In [68]:
# train SVM on player combat -- both
svms_pcombat = train_svms(1, 11, X_pcombat_split)
svms_pcombat_results = print_avgs(svms_pcombat)

Avg misclassified examples: 56
Avg accuracy: 0.793


In [69]:
# train SVM on spell casts
svms_spells = train_svms(1, 11, X_spells_split)
svms_spells_results = print_avgs(svms_spells)

Avg misclassified examples: 110
Avg accuracy: 0.596


In [70]:
# train SVM on kda/killing sprees
svms_kda = train_svms(1, 11, X_kda_split)
svms_kda_results = print_avgs(svms_kda)

Avg misclassified examples: 35
Avg accuracy: 0.870


In [71]:
# train random forest on vision
svms_vision = train_svms(1, 11, X_vision_split)
svms_vision_results = print_avgs(svms_vision)

Avg misclassified examples: 92
Avg accuracy: 0.662


### SVM results

In [72]:
svm_results_list = [svms_all_results, svms_pcombatOff_results, svms_pcombatDef_results, 
                  svms_pcombat_results, svms_spells_results, svms_kda_results, svms_vision_results]

svm_results = pd.DataFrame(svm_results_list, index = features_about, columns = ["Avg misclassified examples", "Avg accuracy"])
svm_results = svm_results.sort_values(by=["Avg accuracy"], ascending=False)
svm_results

,Avg misclassified examples,Avg accuracy
All features,19.21,0.929634
KDA/kill streaks,35.54,0.869817
Player combat,56.39,0.793443
Player combat (defensive),65.23,0.761062
Vision,92.35,0.661722
Player combat (offensive),104.85,0.615934
Spells casted,110.42,0.595531


### Summary of results

The results from all 5 different models (perceptron, logistic regression, decision tree, random forest, SVM) are shown below and are sorted by average accuracy.

In [76]:
# Perceptron
ppn_results.style.set_caption('Perceptron')

,Avg misclassified examples,Avg accuracy
All features,25.540000,0.906447
KDA/kill streaks,51.450000,0.811538
Player combat,78.530000,0.712344
Player combat (defensive),94.950000,0.652198
Vision,118.480000,0.566007
Player combat (offensive),125.990000,0.538498
Spells casted,131.790000,0.517253


In [77]:
# Logistic regression
lrGD_results.style.set_caption('Logistic Regression')

,Avg misclassified examples,Avg accuracy
All features,16.090000,0.941062
KDA/kill streaks,36.550000,0.866117
Player combat,59.440000,0.782271
Player combat (defensive),64.870000,0.762381
Vision,95.800000,0.649084
Player combat (offensive),115.300000,0.577656
Spells casted,117.450000,0.569780


In [78]:
# Decision tree
dt_results.style.set_caption('Decision tree')

,Avg misclassified examples,Avg accuracy
All features,23.000000,0.915751
KDA/kill streaks,26.410000,0.903260
Vision,55.600000,0.796337
Player combat,55.670000,0.796081
Player combat (defensive),56.200000,0.794139
Spells casted,79.210000,0.709853
Player combat (offensive),88.950000,0.674176


In [79]:
# Random forest
rf_results.style.set_caption('Random forest')

,Avg misclassified examples,Avg accuracy
All features,12.200000,0.955311
KDA/kill streaks,23.590000,0.913590
Player combat,30.030000,0.890000
Player combat (defensive),35.160000,0.871209
Vision,44.860000,0.835678
Player combat (offensive),61.530000,0.774615
Spells casted,63.690000,0.766703


In [80]:
# SVM
svm_results.style.set_caption('SVM')

,Avg misclassified examples,Avg accuracy
All features,19.210000,0.929634
KDA/kill streaks,35.540000,0.869817
Player combat,56.390000,0.793443
Player combat (defensive),65.230000,0.761062
Vision,92.350000,0.661722
Player combat (offensive),104.850000,0.615934
Spells casted,110.420000,0.595531


In all models except the decision tree model, the best features to predict the outcome of the game are sorted as follows:

1. All features
2. KDA/kill streaks
3. Player combat (offensive and defensive)
4. Player combat (defensive)
5. Vision
6. Player combat (offensive)
7. Spells casted

In the decision tree model, all features and KDA/kill streaks are consistent with other models in predicting the outcome of the game. Similarly, player combat (offensive) and spells casted are the worst in predicting the outcome of the game.

This is very interesting because overall, League of Legends is a strategy game. The team that is able to gain the most advantages is more likely to win. The results show that aside from using all features, KDA/kill streaks is the best predictor of the outcome of the game. This makes sense because teams with more advantages are more likely to get kills, and kills allow the team to gain even more advantages. On the other hand, the combination of both offensive and defensive player combat statistics is also a rather decent predictor for the outcome of the game. We can also see that defensive player combat statistics is a better predictor than offensive player combat statistics. This may be due to the fact that I mainly play the support role, so I play a defensive style in combat.

### Future work

In the future, it would be interesting to narrow down the data even more and see how different sets of features for a specific character (champion) and role can affect the outcome of the game. It would also be interesting to explore how the accuracy these sets of features compare with different players (such as pro players).